# Product Injury
<font size=4>Data Prep and Load - Codes</font>   
***  

**Project Summary:**   
The Consumer Product Safety Commission operations a surveillance system (NEISS) to track injury data related to consumer products. The data is collected from a representative sample of emergency rooms in the United Status. 
This project will examine the data from 2013 through 2022 to explore trends in product injuries resulting in emergency room visits.

**Notebook Scope:**  
This notebook includes code to validate and cleanup the codes used in the NEISS data file. This data has been downloaded from the [NEISS website](https://www.cpsc.gov/cgibin/NEISSQuery/home.aspx).

**Output:**  
An excel file with a scrubbed list of codes and labels that will be used for analysis of NEISS data.
***  

***
# Notebook Setup
***

In [145]:
# Import libraries
import pandas as pd

***  
# Read Raw Data
***

In [146]:
# Read in the NEIS_FMT worksheet from each workbook
path = '../Data/'
files = ['neiss' + str(x) + '.xlsx' for x in range(2013, 2023)]

all_codes_df = pd.DataFrame()

for file in files:
    raw_df = pd.read_excel(path + file, sheet_name = 'NEISS_FMT')
    raw_df.insert(0, 'Year', file[-9:-5])
    all_codes_df = pd.concat([all_codes_df, raw_df])

all_codes_df

,Year,Format name,Starting value for format,Ending value for format,Format value label
0,2013,AGELTTWO,0,0,UNK
1,2013,AGELTTWO,2,120,2 YEARS AND OLDER
2,2013,AGELTTWO,201,201,1 MONTH
3,2013,AGELTTWO,202,202,2 MONTHS
4,2013,AGELTTWO,203,203,3 MONTHS
...,...,...,...,...,...
1244,2022,RACE,2,2,BLACK/AFRICAN AMERICAN
1245,2022,RACE,3,3,OTHER
1246,2022,RACE,4,4,ASIAN
1247,2022,RACE,5,5,AMERICAN INDIAN/ALASKA NATIVE


In [147]:
# Reset index
all_codes_df.reset_index(drop=True, inplace=True)

In [148]:
# Rename columns for clarity
all_codes_df.columns = ['Year', 'Code', 'Start Value', 'End Value', 'Label']

***  
# Verify Data and Types
***  

In [149]:
# Look for columns with null values
all_codes_df.columns[all_codes_df.isnull().any()]

Index(['Label'], dtype='object')

In [150]:
# Delete all codes that do not contain a label
rows_to_del = all_codes_df[all_codes_df['Label'].isnull()].index
all_codes_df.drop(rows_to_del, axis='index', inplace=True)

In [151]:
# Review data types
all_codes_df.dtypes

Year           object
Code           object
Start Value    object
End Value      object
Label          object
dtype: object

***
## Review Year Data
***

In [152]:
# Verify all Year data is in our present, and in our date range
all_codes_df['Year'].unique()

array(['2013', '2014', '2015', '2016', '2017', '2018', '2019', '2020',
       '2021', '2022'], dtype=object)

In [153]:
# Set Year data type to int
all_codes_df['Year'] = all_codes_df['Year'].astype(int)

***
## Review Format Name Data
***

In [154]:
# Verify all Format name data is valid
all_codes_df['Code'].unique()

array(['AGELTTWO', 'ALC_DRUG', 'BDYPT', 'DIAG', 'DISP', 'FIRE', 'GENDER',
       'HISP', 'LOC', 'PROD', 'RACE'], dtype=object)

***
## Review Starting Value Data
***

In [155]:
# Review Starting Value data
all_codes_df['Start Value'].unique()

array(['               0', '               2', '             201', ...,
       '            9999', '             714', '            1552'],
      dtype=object)

In [156]:
# Strip leading spaces and check that the remaining data is numeric
all_codes_df['Start Value'] = all_codes_df['Start Value'].str.strip()
all_codes_df[~all_codes_df['Start Value'].str.isnumeric()]

,Year,Code,Start Value,End Value,Label
25,2013,ALC_DRUG,.,.,NA before 2019
105,2013,HISP,.,.,NA before 2019
1273,2014,ALC_DRUG,.,.,NA before 2019
1353,2014,HISP,.,.,NA before 2019
2521,2015,ALC_DRUG,.,.,NA before 2019
2601,2015,HISP,.,.,NA before 2019
3769,2016,ALC_DRUG,.,.,NA before 2019
3849,2016,HISP,.,.,NA before 2019
5017,2017,ALC_DRUG,.,.,NA before 2019
5097,2017,HISP,.,.,NA before 2019


In [157]:
# Remove rows with Starting value for format of period and convert column type to int
rows_to_del = all_codes_df[~all_codes_df['Start Value'].str.isnumeric()].index
all_codes_df.drop(rows_to_del, axis='index', inplace=True)
all_codes_df['Start Value'] = all_codes_df['Start Value'].astype(int)

***
## Review Ending Value Data
***

In [158]:
# Review Ending Value data
all_codes_df['End Value'].unique()

array(['               0', '             120', '             201', ...,
       '            9999', '             714', '            1552'],
      dtype=object)

In [159]:
# Strip leading spaces and check that the remaining data is numeric
all_codes_df['End Value'] = all_codes_df['End Value'].str.strip()
all_codes_df[~all_codes_df['End Value'].str.isnumeric()]

,Year,Code,Start Value,End Value,Label


In [160]:
# Convert Ending value data to int
all_codes_df['End Value'] = all_codes_df['End Value'].astype(int)

***
## Review Label Data
***

In [161]:
# Review labels
all_codes_df['Label'].unique()

array(['UNK', '2 YEARS AND OLDER', '1 MONTH', ...,
       '714 - COMBINATION FIRE/SMOKE ALARM AND CARBON MONOXIDE DETECTORS',
       'NON-BINARY/OTHER', '1552 - CRIBS, NONPORTABLE OR NOT SPECIFIED'],
      dtype=object)

In [162]:
# Refresh our list of Format Name values
all_codes_df['Code'].unique()

array(['AGELTTWO', 'ALC_DRUG', 'BDYPT', 'DIAG', 'DISP', 'FIRE', 'GENDER',
       'HISP', 'LOC', 'PROD', 'RACE'], dtype=object)

In [163]:
# Since there are so many labels, let's review by Format Name
all_codes_df[all_codes_df['Code'] == 'AGELTTWO']['Label'].unique()

array(['UNK', '2 YEARS AND OLDER', '1 MONTH', '2 MONTHS', '3 MONTHS',
       '4 MONTHS', '5 MONTHS', '6 MONTHS', '7 MONTHS', '8 MONTHS',
       '9 MONTHS', '10 MONTHS', '11 MONTHS', '12 MONTHS', '13 MONTHS',
       '14 MONTHS', '15 MONTHS', '16 MONTHS', '17 MONTHS', '18 MONTHS',
       '19 MONTHS', '20 MONTHS', '21 MONTHS', '22 MONTHS', '23 MONTHS'],
      dtype=object)

In [164]:
# Review labels by Alcohol / Drug Involvement
all_codes_df[all_codes_df['Code'] == 'ALC_DRUG']['Label'].unique()

array(['No/Unk', 'Yes'], dtype=object)

In [165]:
# Review labels by Body Part
all_codes_df[all_codes_df['Code'] == 'BDYPT']['Label'].unique()

array(['0 - INTERNAL', '30 - SHOULDER', '31 - UPPER TRUNK', '32 - ELBOW',
       '33 - LOWER ARM', '34 - WRIST', '35 - KNEE', '36 - LOWER LEG',
       '37 - ANKLE', '38 - PUBIC REGION', '75 - HEAD', '76 - FACE',
       '77 - EYEBALL', '78 - UPPER TRUNK(OLD)', '79 - LOWER TRUNK',
       '80 - UPPER ARM', '81 - UPPER LEG', '82 - HAND', '83 - FOOT',
       '84 - 25-50% OF BODY', '85 - ALL PARTS BODY', '86 - OTHER(OLD)',
       '87 - NOT STATED/UNK', '88 - MOUTH', '89 - NECK',
       '90 - LOWER ARM(OLD)', '91 - LOWER LEG(OLD)', '92 - FINGER',
       '93 - TOE', '94 - EAR'], dtype=object)

In [166]:
# Review labels by Diagnosis
all_codes_df[all_codes_df['Code'] == 'DIAG']['Label'].unique()

array(['41 - INGESTION', '42 - ASPIRATION', '46 - BURN, ELECTRICAL',
       '47 - BURN, NOT SPEC.', '48 - BURN, SCALD', '49 - BURN, CHEMICAL',
       '50 - AMPUTATION', '51 - BURNS, THERMAL', '52 - CONCUSSION',
       '53 - CONTUSIONS, ABR.', '54 - CRUSHING', '55 - DISLOCATION',
       '56 - FOREIGN BODY', '57 - FRACTURE', '58 - HEMATOMA',
       '59 - LACERATION', '60 - DENTAL INJURY', '61 - NERVE DAMAGE',
       '62 - INTERNAL INJURY', '63 - PUNCTURE', '64 - STRAIN, SPRAIN',
       '65 - ANOXIA', '66 - HEMORRHAGE', '67 - ELECTRIC SHOCK',
       '68 - POISONING', '69 - SUBMERSION', '70 - OTHER', '71 - OTHER',
       '72 - AVULSION', '73 - RADIATION', '74 - DERMA/CONJUNCT'],
      dtype=object)

In [167]:
# Review labels by Disosition
all_codes_df[all_codes_df['Code'] == 'DISP']['Label'].unique()

array(['0 - NO INJURY', '1 - TREATED/EXAMINED AND RELEASED',
       '2 - TREATED AND TRANSFERRED',
       '4 - TREATED AND ADMITTED/HOSPITALIZED',
       '5 - HELD FOR OBSERVATION', '6 - LEFT WITHOUT BEING SEEN',
       '8 - FATALITY INCL. DOA, DIED IN ER', '9 - UNKNOWN, NOT STATED'],
      dtype=object)

In [168]:
# Review labels by Fire Involvement
all_codes_df[all_codes_df['Code'] == 'FIRE']['Label'].unique()

array(['NO/?', 'FD', 'NO FD', '? FD', 'MV'], dtype=object)

In [169]:
# Review labels by Gender
all_codes_df[all_codes_df['Code'] == 'GENDER']['Label'].unique()

array(['UNKNOWN', 'MALE', 'FEMALE', 'NON-BINARY/OTHER'], dtype=object)

In [170]:
# Review labels by Gender
all_codes_df[all_codes_df['Code'] == 'HISP']['Label'].unique()

array(['Unk/Not stated', 'Yes', 'No'], dtype=object)

In [171]:
# Review labels by Location
all_codes_df[all_codes_df['Code'] == 'LOC']['Label'].unique()

array(['UNK', 'HOME', 'FARM', 'APART.', 'STREET', 'PUBLIC', 'MOBILE',
       'INDST.', 'SCHOOL', 'SPORTS'], dtype=object)

In [172]:
# Review labels by Product
all_codes_df[all_codes_df['Code'] == 'PROD']['Label'].unique()

array(['101 - WASHING MACHINES WITHOUT WRINGERS OR OTHER DRYERS',
       '102 - WRINGER WASHING MACHINES',
       '103 - WASHING MACHINES WITH UNHEATED SPIN DRYERS', ...,
       '9999 - UNCATEGORIZED PRODUCT',
       '714 - COMBINATION FIRE/SMOKE ALARM AND CARBON MONOXIDE DETECTORS',
       '1552 - CRIBS, NONPORTABLE OR NOT SPECIFIED'], dtype=object)

In [173]:
# Review labels by Race
all_codes_df[all_codes_df['Code'] == 'RACE']['Label'].unique()

array(['N.S.', 'WHITE', 'BLACK/AFRICAN AMERICAN', 'OTHER', 'ASIAN',
       'AMERICAN INDIAN/ALASKA NATIVE',
       'NATIVE HAWAIIAN/PACIFIC ISLANDER'], dtype=object)

***
# Data Consistency
***

In [174]:
# Several Labels include a code prefix. For consistency, let's strip those
def strip_code(label):
    if ' - ' in label:
        return label.split(' - ')[1]
    else:
        return label

all_codes_df['Label'] = all_codes_df['Label'].apply(strip_code)

In [175]:
# Verify that a all code categories are present in each annual set of data
missing_cats = 0
for cat in all_codes_df['Code'].unique():
    for yr in range(2013, 2023):
        if len(all_codes_df[all_codes_df['Code'] == cat]) == 0:
            mssing_cats += 1
            print(f'{yr}: Missing {cat}')
if missing_cats == 0:
    print('Validation check passed - all categories are included in all years.')

Validation check passed - all categories are included in all years.


In [179]:
# Let's check for duplicate starting values by year and category
counts_df = all_codes_df.groupby(['Year', 'Code', 'Start Value']).count()
dups_df = counts_df.loc[counts_df['End Value'] > 1]
if len(dups_df) > 1:
    display(dups_df)
else:
    print('There are no duplicate starting values for amy codes for any year.')

There are no duplicate starting values for amy codes for any year.


In [180]:
temp_df['Year'].count()

10

In [181]:
# Since there are no duplicate starting values within a given category for a given year, we will use a combination of year, format
# and starting value to look for any inconsistencies in codes and labels across the dataset
mismatches = False
for cat in all_codes_df['Code'].unique():
    start_vals = all_codes_df[all_codes_df['Code'] == cat]['Start Value'].unique()
    for val in start_vals:
        temp_df = all_codes_df[(all_codes_df['Code'] == cat) & (all_codes_df['Start Value'] == val)]
        if len(temp_df['End Value'].unique()) > 1 or len(temp_df['Label'].unique()) > 1:
            display(temp_df)
            mismatches = True
        if temp_df['Year'].count() != 10:
            print(f'{cat} code {val} is not included in all years.')
            mismatches = True   

if mismatches == False:
    print('All codes and labels are consistent across years.')

GENDER code 3 is not included in all years.
PROD code 714 is not included in all years.
PROD code 1552 is not included in all years.


In [182]:
# Review codes that use a range of values (start value != end value)
all_codes_df[all_codes_df['Start Value'] != all_codes_df['End Value']]

,Year,Code,Start Value,End Value,Label
1,2013,AGELTTWO,2,120,2 YEARS AND OLDER
1249,2014,AGELTTWO,2,120,2 YEARS AND OLDER
2497,2015,AGELTTWO,2,120,2 YEARS AND OLDER
3745,2016,AGELTTWO,2,120,2 YEARS AND OLDER
4993,2017,AGELTTWO,2,120,2 YEARS AND OLDER
6241,2018,AGELTTWO,2,120,2 YEARS AND OLDER
7489,2019,AGELTTWO,2,120,2 YEARS AND OLDER
8737,2020,AGELTTWO,2,120,2 YEARS AND OLDER
9986,2021,AGELTTWO,2,120,2 YEARS AND OLDER
11235,2022,AGELTTWO,2,120,2 YEARS AND OLDER


***
# Create Final Code Table
***

In [191]:
# Drop the 2 YEARS AND OLDER row, as they the value and the code are the same (no translation needed)
rows_to_drop = all_codes_df[all_codes_df['Label'] == '2 YEARS AND OLDER'].index
std_codes_df = all_codes_df.drop(rows_to_drop, axis='index').copy()

In [192]:
# Get rid of unnedded columns and rename for clarity
std_codes_df = std_codes_df[['Code', 'Start Value', 'Label']].drop_duplicates()

In [194]:
# Rename start value column
std_codes_df.rename(columns={'Start Value': 'Value'}, inplace=True)

In [195]:
std_codes_df

,Code,Value,Label
0,AGELTTWO,0,UNK
2,AGELTTWO,201,1 MONTH
3,AGELTTWO,202,2 MONTHS
4,AGELTTWO,203,3 MONTHS
5,AGELTTWO,204,4 MONTHS
...,...,...,...
1246,RACE,5,AMERICAN INDIAN/ALASKA NATIVE
1247,RACE,6,NATIVE HAWAIIAN/PACIFIC ISLANDER
9113,PROD,714,COMBINATION FIRE/SMOKE ALARM AND CARBON MONOXI...
10090,GENDER,3,NON-BINARY/OTHER


In [197]:
# Write final tabel to Excel
std_codes_df.to_excel('../Data/Standard Codes.xlsx', index=False)

***
**End**
***